In [1]:
import pandas as pd
import numpy as np

In [2]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [150]:
def get_summary(scenario, _set = None):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    
    
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values

    #prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    #prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['my_score'] = prediction.score
    prediction['base_score'] = prediction.base_score

    if _set is None:
        sset_prediction = prediction
    else:
        sset_prediction = prediction[prediction._set == _set]
    
    #accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    scores = (sset_prediction[['my_score','base_score']].sum()/sset_prediction.shape[0]).values


    #print('acc', accs, accs[0] > accs[1])
    print('scores', scores, scores[0] > scores[1])


    accurracies = sset_prediction[['q_type', 'my_score','base_score',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
                                # 'my_acc','base_acc', 
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    #new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    #new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    new_summary['my_score'] = accurracies.loc[new_summary._q_type.values].my_score.values
    new_summary['base_score'] =  accurracies.loc[new_summary._q_type.values].base_score.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['above_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_score','base_score']].sum() #'my_acc','base_acc',
    
    #print('acc_sum', _sum.values[:2], _sum[0] > _sum[1])#/accurracies.shape[0]
    print('score_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]


    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    new_summary['diff_score'] = (new_summary.my_score - new_summary.base_score)

    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_top_down']
    
    #del new_summary['base_acc']
    #del new_summary['my_acc']



    print(new_summary._count.sum())
    return new_summary

In [151]:
def get_differents(summary, p=1, min_count=2):
    #summary = summary[summary._count >= min_count]
    
    mean_dif = summary._diff.mean()
    std_dif = summary._diff.std()
    data_dif = summary._diff
    data_diff = pd.DataFrame( ((data_dif < mean_dif - p * std_dif).astype(int) + 
                             (data_dif > mean_dif + p * std_dif).astype(int)) > 0 )
    significantly_different = summary.loc[data_diff[data_diff._diff].index]
    print(data_diff.shape, 'shape')
    
    different = significantly_different[significantly_different._count>min_count].sort_values(by='winner')
    

    different = different[different.diff_score!=0]

    print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 3),'% query_types')
    print(np.round(float(different._count.sum())/ summary._count.sum() * 100, 3),'% queries')
    
    vals = different[['my_score','base_score']].sum().values
    print(vals, vals[0]>vals[1], 'sum_scores')
    print(int(different.diff_score.sum()), 'abs diff')
    
    print(different.shape[0],'len')
    different.sort_values(by=['winner','diff_score'], inplace=True, ascending=[True, False])
    return different

# Real

In [164]:
scenario = 'real'
summary = get_summary(scenario, _set='val')
print('\n')
different = get_differents(summary, p=1, min_count=2)
different

('scores', array([ 0.63612855,  0.63371339]), True)
('score_sum', array([ 136356.7,  135839. ]), True)
214354


((317, 1), 'shape')
(11.041, '% query_types')
(11.644, '% queries')
(array([ 18095.2,  18012.5]), True, 'sum_scores')
(82, 'abs diff')
(35, 'len')


,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
391,what color is the,0.514800,0.5,14073,True,bottom_up,11018.9,10946.8,True,-0.029601,72.1
389,what color are the,0.517062,0.5,3221,True,bottom_up,2426.4,2419.0,True,-0.034123,7.4
485,who many,0.514453,0.5,18,True,bottom_up,7.2,4.2,True,-0.028907,3.0
253,name on,0.518880,0.5,7,True,bottom_up,6.0,3.6,False,-0.037761,2.4
501,who rides,0.524430,0.5,7,True,bottom_up,5.0,4.0,False,-0.048860,1.0
373,urban or,0.526171,0.5,14,True,bottom_up,14.0,13.0,True,-0.052343,1.0
570,will those,0.524738,0.5,3,True,bottom_up,2.0,1.0,False,-0.049476,1.0
324,sunny or,0.536791,0.5,50,True,bottom_up,42.9,42.4,True,-0.073581,0.5
422,where might,0.521554,0.5,12,True,bottom_up,7.4,7.3,True,-0.043107,0.1
384,what animal is,0.517966,0.5,836,True,bottom_up,620.9,621.0,True,-0.035931,-0.1


In [165]:
summary[summary.winner=='bottom_up'].sort_values(by='diff_score', ascending=False)[:20]

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
210,is the,0.503316,0.5,16604,True,bottom_up,12841.4,12752.9,True,-0.006632,88.5
202,in the,0.510852,0.5,5785,True,bottom_up,3502.2,3419.2,True,-0.021704,83.0
391,what color is the,0.514800,0.5,14073,True,bottom_up,11018.9,10946.8,True,-0.029601,72.1
383,what,0.508178,0.5,16197,True,bottom_up,7507.8,7440.3,True,-0.016357,67.5
188,how many,0.512289,0.5,13779,True,bottom_up,6815.4,6753.7,True,-0.024578,61.7
189,how many people are,0.507203,0.5,1780,True,bottom_up,981.4,948.9,True,-0.014405,32.5
435,which,0.508893,0.5,2554,True,bottom_up,1159.3,1127.2,True,-0.017786,32.1
408,what type of,0.509495,0.5,4061,True,bottom_up,2299.7,2272.2,True,-0.018989,27.5
206,is,0.502419,0.5,12545,True,bottom_up,8473.9,8450.6,True,-0.004837,23.3
212,is the person,0.501203,0.5,808,True,bottom_up,610.5,589.7,True,-0.002406,20.8


In [166]:
summary[summary.winner=='top_down'].sort_values(by='diff_score', ascending=False)[:20]

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
211,is the man,0.497037,0.5,2699,True,top_down,2114.2,2080.3,True,0.005925,33.9
141,does the,0.497173,0.5,2499,True,top_down,1940.2,1916.5,True,0.005655,23.7
401,what is the woman,0.479126,0.5,875,True,top_down,505.2,489.4,True,0.041748,15.8
405,what room is,0.490784,0.5,765,True,top_down,709.6,694.0,True,0.018433,15.6
139,do,0.499391,0.5,2651,True,top_down,2041.0,2028.4,True,0.001218,12.6
231,man or,0.483317,0.5,228,True,top_down,195.7,185.4,True,0.033365,10.3
402,what is this,0.496490,0.5,1694,True,top_down,1080.7,1071.4,True,0.007019,9.3
207,is he,0.497805,0.5,1261,True,top_down,967.9,960.0,True,0.004390,7.9
400,what is the person,0.484777,0.5,910,True,top_down,590.4,582.5,True,0.030446,7.9
131,did the,0.496037,0.5,213,True,top_down,175.3,168.3,True,0.007926,7.0


# Abstract

In [153]:
scenario = 'abstract'
summary = get_summary(scenario, _set='val')
print('\n')
different = get_differents(summary, p=.66, min_count=2)
different

('scores', array([ 0.71153   ,  0.71069333]), True)
('score_sum', array([ 21345.9,  21320.8]), True)
30000


((177, 1), 'shape')
(16.949000000000002, '% query_types')
(31.382999999999999, '% queries')
(array([ 6006.1,  5978.8]), True, 'sum_scores')
(27, 'abs diff')
(30, 'len')


,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
43,do,0.500101,0.5,330,True,bottom_up,274.6,267.3,True,-0.000202,7.3
7,are there any,0.520157,0.5,172,True,bottom_up,152.9,152.0,True,-0.040314,0.9
102,is the tv,0.502234,0.5,101,True,bottom_up,89.0,88.4,True,-0.004467,0.6
86,is anyone,0.515345,0.5,95,True,bottom_up,89.2,89.3,True,-0.030691,-0.1
221,will that,0.502367,0.5,3,True,bottom_up,2.6,3.0,False,-0.004735,-0.4
227,would a,0.506128,0.5,3,True,bottom_up,0.3,1.3,False,-0.012255,-1.0
164,what color is,0.506299,0.5,81,True,bottom_up,71.5,72.9,True,-0.012599,-1.4
101,is the sun,0.500762,0.5,155,True,bottom_up,128.7,132.2,True,-0.001523,-3.5
65,have the,0.501312,0.5,44,True,bottom_up,26.9,30.9,True,-0.002625,-4.0
162,what color,0.500193,0.5,164,True,bottom_up,132.7,137.8,True,-0.000387,-5.1


In [162]:
summary[summary.winner=='bottom_up'].sort_values(by='diff_score', ascending=False)[:20]

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
43,do,0.500101,0.5,330,True,bottom_up,274.6,267.3,True,-0.000202,7.3
204,will all,0.502450,0.5,1,False,bottom_up,1.0,0.0,False,-0.004900,1.0
196,where's the,0.535325,0.5,1,False,bottom_up,1.0,0.0,False,-0.070650,1.0
7,are there any,0.520157,0.5,172,True,bottom_up,152.9,152.0,True,-0.040314,0.9
102,is the tv,0.502234,0.5,101,True,bottom_up,89.0,88.4,True,-0.004467,0.6
2,any coats,0.518402,0.5,1,False,bottom_up,1.0,1.0,False,-0.036804,0.0
232,would more,0.520383,0.5,1,False,bottom_up,0.0,0.0,False,-0.040766,0.0
211,will it,0.506267,0.5,15,True,bottom_up,12.0,12.0,True,-0.012534,0.0
207,will anyone,0.501637,0.5,1,False,bottom_up,1.0,1.0,False,-0.003274,0.0
195,where would,0.543903,0.5,1,False,bottom_up,0.3,0.3,False,-0.087807,0.0


In [163]:
summary[summary.winner=='top_down'].sort_values(by='diff_score', ascending=False)[:20]

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
69,how many,0.471090,0.5,2686,True,top_down,1568.6,1506.6,True,0.057820,62.0
99,is the man,0.493456,0.5,580,True,top_down,484.6,470.4,True,0.013089,14.2
103,is the woman,0.494635,0.5,687,True,top_down,561.1,548.6,True,0.010731,12.5
71,how many birds are,0.468498,0.5,86,True,top_down,43.5,32.7,True,0.063004,10.8
70,how many animals are,0.467302,0.5,86,True,top_down,41.3,31.0,True,0.065396,10.3
6,are there,0.499357,0.5,237,True,top_down,202.1,194.1,True,0.001287,8.0
46,does the,0.494442,0.5,625,True,top_down,516.1,508.9,True,0.011116,7.2
3,are,0.495761,0.5,177,True,top_down,137.1,130.2,True,0.008477,6.9
72,how many bushes,0.462417,0.5,87,True,top_down,46.7,40.2,True,0.075166,6.5
165,what color is the,0.498515,0.5,2098,True,top_down,1898.8,1893.3,True,0.002969,5.5
